In [3]:
import numpy as np, re
from itertools import cycle
from aoclib import list2string, string2list
from itertools import combinations

lines = open('20.txt').read().strip()

In [252]:
segments = lines.split("\n\n")
tileids = [segment.splitlines()[0].strip("Tile ") for segment in segments]
grids = np.array([[list(x) for x in segment.splitlines()[1:]] for segment in segments])

dirs = ("up", "right", "down", "left")
edges = dict()
for i, x in enumerate(grids):
    edges[tileids[i] + dirs[0]], edges[tileids[i] + dirs[1]], edges[tileids[i] + dirs[2]], edges[tileids[i] + dirs[3]] = x[0], x[:, -1], x[-1], x[:, 0]
    edges[tileids[i] + dirs[0] + "R"], edges[tileids[i] + dirs[1] + "R"], edges[tileids[i] + dirs[2] + "R"], edges[tileids[i] + dirs[3] + "R"] = \
        np.flip(x[0]), np.flip(x[:, -1]), np.flip(x[-1]), np.flip(x[:, 0])

congruent = []
for comb in combinations(edges, 2):
    if all(edges[comb[0]] == edges[comb[1]]):
        congruent.append(comb)
#print(congruent)
print(len(congruent))

# Part 1
adj_tiles = set()
puzzles = []
for border1, border2 in congruent:
    border1, border2 = border1.split(":"), border2.split(":")
    if (border1[0], border2[0]) not in adj_tiles:
        adj_tiles.add((border1[0], border2[0]))
        puzzles.append((tuple(border1), tuple(border2)))

cornerids = []
result = 1
for tileid in tileids:
    counter = 0
    for pair in adj_tiles:
        if tileid.strip(":") in pair:
            counter += 1
    if counter == 2:
        cornerids.append(tileid.strip(":"))
        result *= int(cornerids[-1])
print(result)

528
45079100979683


In [350]:
import sys, time

tileids = [x.strip(":") for x in tileids]
grids_dict = dict()
for t, g in zip(tileids, grids):
    grids_dict[t] = g

transformations = [lambda x: x, lambda x: np.rot90(x), lambda x: np.rot90(x, k=2), lambda x: np.rot90(x, k=3), \
                    lambda x: np.fliplr(x), lambda x: np.rot90(np.fliplr(x)), lambda x: np.rot90(np.fliplr(x), k=2), lambda x: np.rot90(np.fliplr(x), k=3)]

starttime = time.time()

for line in adj_tiles:
#    print(line)
    pass

adj_tiles_1dlist = [x for line in adj_tiles for x in line]
adj_tiles_2dlist = [list(line) for line in adj_tiles]

# Find corner tile
for tileid in tileids:
    counter = adj_tiles_1dlist.count(tileid)
    if counter == 2:
        break
picture = [['.' for i in range(12)] for j in range(12)]
lefttiles = []
y, x = 0, 0

# First step
for i, p in enumerate(adj_tiles_2dlist):
    if tileid in p:
        while p in adj_tiles_2dlist:
            adj_tiles_2dlist.remove(p)
        prev = list(reversed(p))
        while list(reversed(p)) in adj_tiles_2dlist:
            adj_tiles_2dlist.remove(prev)
        lefttiles.append(tileid)
        break
picture[y][x] = grids_dict[tileid]
picture[y][x + 1] = grids_dict[p[1 - p.index(tileid)]]

order = [tileid, p[1 - p.index(tileid)]]
for puz in puzzles:
    if puz[0][0] == '2251':# and puz[1][0] == p[1]:
        print(puz)
print(i, p)

#print(picture)
#print(picture[y][x])
for T1 in transformations:
    flag = False
    for T2 in transformations:
        #print(T1(picture[y][x])[:, -1], T2(picture[y][x + 1])[:, 0])
        #print(picture[y][x + 1])
        #print(picture[y][x + 1][:, 0])
        t_spot1 = T1(picture[y][x])
        t_spot2 = T2(picture[y][x + 1])
        if all(t_spot1[:, -1] == t_spot2[:, 0]):
            print('match', transformations.index(T1), transformations.index(T2))
            picture[y][x] = np.flipud(t_spot1)
            picture[y][x + 1] = np.flipud(t_spot2)
            flag = True
            break
    if flag:
        break
tileid = p[1 - p.index(tileid)]
print(tileid)

# Further steps
def finish_row(y, x, starttile):
    tileid = starttile
    while x < len(picture) - 1:
        for i, p in enumerate(adj_tiles_2dlist):
            flag = False
            if tileid in p:
                #print(i)
                idx = 1 - p.index(tileid)
                #print(p, tileid)
                for T in transformations:
                    #print(y, x)
                    picture[y][x][:, -1]
                    #print(grids_dict[p[idx]])
                    if all(picture[y][x][:, -1] == T(grids_dict[p[idx]])[:, 0]):
                        if y > 0:
                            assert all(picture[y-1][x+1][-1, :] == T(grids_dict[p[idx]])[0, :])
                        order.append(p[idx])
                        #print('match2')
                        picture[y][x + 1] = T(grids_dict[p[idx]])
                        while p in adj_tiles_2dlist:
                            #print("rem")
                            adj_tiles_2dlist.remove(p)
                        prev = list(reversed(p))
                        while prev in adj_tiles_2dlist:
                            #print("rem2")
                            adj_tiles_2dlist.remove(prev)
                        #print(picture)
                        tileid = p[idx]
                        #print(tileid, idx, p)
                        flag = True
                        break

            if flag:
                flag = False
                break
        #print('x increased')
        x += 1
    return starttile

tileid = finish_row(0, 1, tileid)

for y in range(0, len(picture) - 1):
    tileid = lefttiles[-1]
    #print(tileid)
    to_match = picture[y][0][-1, :]
    for i, p in enumerate(adj_tiles_2dlist):
        flag = False
        if tileid in p:
            #print("=======")
            #print(i)
            idx = 1 - p.index(tileid)
            #print(p, tileid)
            for T in transformations:
                #print(y, x)
                #picture[y][x][:, -1]
                #print(grids_dict[p[idx]])
                if all(to_match == T(grids_dict[p[idx]])[0, :]):
                    order.append(p[idx])
                    #print('match3')
                    picture[y + 1][0] = T(grids_dict[p[idx]])
                    while p in adj_tiles_2dlist:
                        #print("rem")
                        adj_tiles_2dlist.remove(p)
                    prev = list(reversed(p))
                    while prev in adj_tiles_2dlist:
                        #print("rem2")
                        adj_tiles_2dlist.remove(prev)
                    #print(picture)
                    lefttiles.append(p[idx])
                    #print(tileid, idx, p)
                    flag = True
                    break
                    
        if flag:
            flag = False
            break
    #print('y increased')
    finish_row(y + 1, 0, lefttiles[-1])
    
print(time.time() - starttime, "")

for i, row in enumerate(picture):
    for j, cell in enumerate(row):
        picture[i][j] = cell[1:-1, 1:-1]

pic = np.concatenate((picture[0][:]), axis = 1)
for y in range(1, 12):
    pic = np.concatenate((pic, np.concatenate((picture[y][:]), axis = 1)), axis = 0)

def np2str(x):
    rows = []
    for i in range(x.shape[0]):
        rows.append(''.join(x[i, :]))
    str_ = '\n'.join(rows)
    return str_

print(order.index('3637'))
#print(np2str(pic))

kernel=["                  # ",
            "#    ##    ##    ###",
            " #  #  #  #  #  #   "]
kernel = ['                  # ', '#    ##    ##    ###', ' #  #  #  #  #  #   ']
kernel = [x.replace(" ", ".") for x in kernel]
kernel_re = [x.replace(" ", "[ ]{1}") for x in kernel]

for T in transformations:
    transformed_pic = T(pic.copy())
    picrows = np2str(transformed_pic).splitlines()
    for i in range(len(picrows)-2):
        pos1 = str.find(picrows[i], kernel[0])
        if pos1 != -1:
            pos2 = str.find(picrows[i+1], kernel[1])
            if pos1 == pos2:
                pos3 = str.find(picrows[i+2], kernel[2])
                if pos2 == pos3:
                        print(pos1, pos2, pos3)
           # except: pass

monster_locations = []
for j, T in enumerate(transformations):
    transformed_pic = T(pic)
    picrows = np2str(transformed_pic).splitlines()
    for i in range(len(picrows)-2):
        matches = re.finditer(r"(?=(" + kernel_re[2] + "))", picrows[i+2])
        while m := next(matches, False):
            x1, _ = m.span()
            print("potential monster 1 row", i, x1)
            matches2 = re.finditer(r"(?=(" + kernel_re[1] + "))", picrows[i+1])
            while True:
                m2 = next(matches2, False)
                if not m2:
                    break
                x2, _ = m2.span()
                print("potential monster 2 rows", i, x2)
                if x1 == x2:
                    match = 1
                    for l in range(len(kernel_re[0])):
                        if kernel_re[0][l] == '#':
                            if picrows[i][x1 + l] != '#':
                                match = 0
                                break
                    if match:
                        print("monster found for trafo", j, "in row", i, "at x =", x1, x2, x3)
                        monster_locations.append((i, x1))
                        correct_trafo = transformed_pic
    if monster_locations:
        break

ownresult = np2str(transformed_pic)
transformed_pic = string2list(np2str(transformed_pic))
print(monster_locations)
for loc in monster_locations:
    for y, kernelrow in zip(range(loc[0], loc[0] + 3), kernel):
        for x, k in zip(range(loc[1], loc[1] + len(kernelrow)), kernelrow):
            if k == '#':
                transformed_pic[y][x] = 'O'
            #if transformed_pic[y][x] == '#':
            #    transformed_pic[y][x] = '-'

print(list2string(transformed_pic).count('#'))
print(len(transformed_pic), len(transformed_pic[0]))
#print(list2string(transformed_pic))

(('2251', 'up'), ('1553', 'down'))
(('2251', 'left'), ('3637', 'right'))
31 ['2251', '1553']
match 3 3
1553
0.012423992156982422 
12
potential monster 1 row 28 62
potential monster 1 row 45 13
potential monster 1 row 3 42
potential monster 1 row 4 38
potential monster 2 rows 4 38
monster found for trafo 1 in row 4 at x = 38 38 74
potential monster 1 row 4 41
potential monster 2 rows 4 38
potential monster 1 row 4 44
potential monster 2 rows 4 38
potential monster 1 row 5 12
potential monster 2 rows 5 12
monster found for trafo 1 in row 5 at x = 12 12 74
potential monster 1 row 10 48
potential monster 2 rows 10 48
monster found for trafo 1 in row 10 at x = 48 48 74
potential monster 1 row 15 3
potential monster 2 rows 15 3
monster found for trafo 1 in row 15 at x = 3 3 74
potential monster 2 rows 15 9
potential monster 1 row 19 44
potential monster 2 rows 19 44
monster found for trafo 1 in row 19 at x = 44 44 74
potential monster 1 row 26 68
potential monster 2 rows 26 74
potential mons

In [301]:
#! /usr/bin/python3

# https://adventofcode.com/2020/day/20

f = open('20.txt', 'r')
#f = open('sample', 'r')

def reverse(num):
    result = 0
    for i in range(10):
        result = (result << 1) + (num & 1)
        num >>= 1
    return result
    
tiles = []
TLEN = 10
#TNUM = 3
TNUM = 12

while line := f.readline():
	if line[0] == 'T':
		num = int(line[5:len(line) - 2])
		tile = []
		while (line := f.readline().rstrip()) != '':
			line = line.translate(line.maketrans('.#', '01'))
			tile.append(line)
		up = int(tile[0], base=2)
		down = int(tile[TLEN - 1], base=2)
		right = int(''.join([tile[i][TLEN - 1] for i in range(TLEN)]), base=2)
		left = int(''.join([tile[i][0] for i in range(TLEN)]), base=2)
		up_rev = reverse(up)
		down_rev = reverse(down)
		right_rev = reverse(right)
		left_rev = reverse(left)
		tiles.append([(up, right, down, left), (left_rev, up, right_rev, down), (down_rev, left_rev, up_rev, right_rev), (right, down_rev, left, up_rev), (up_rev, left, down_rev, right), (right_rev, up_rev, left_rev, down_rev), (down, right_rev, up, left_rev), (left, down, right, up), set((up, down, right, left, up_rev, down_rev, right_rev, left_rev)), num, set(), tile])

for i in range(len(tiles)):
	for j in range(i + 1, len(tiles)):
		if tiles[i][8] & tiles[j][8]:
			tiles[i][10].add(j)
			tiles[j][10].add(i)

def rotate(grid):
	l = len(grid)
	new_grid = [['' for i in range(l)] for j in range(l)]
	for i in range(l):
		for j in range(l):
			new_grid[j][l - 1 - i] = grid[i][j]
	return new_grid

def flip(grid):
	l = len(grid)
	for i in range(l):
		grid[i].reverse()
	return grid		

def dump(grid):
	for i in range(len(grid)):
		for j in range(len(grid[0])):
			print(grid[i][j], end='')
		print()
	print()
	
def analyze(square):
	length = TNUM * (TLEN - 2)
	image = [['' for i in range(length)] for j in range(length)]
	for i in range(TNUM):
		for j in range(TNUM):
			tile = tiles[square[i][j]['tile']][11]
			tile.pop()
			tile.pop(0)
			for k in range(TLEN - 2):
				tile[k] = list(tile[k][1:TLEN-1])
			orient = square[i][j]['orient']
			if orient > 3:
				tile = flip(tile)
				orient -= 4
			for k in range(orient):
				tile = rotate(tile)
			for k in range(TLEN - 2):
				for l in range(TLEN - 2):
					image[i * (TLEN - 2) + k][j * (TLEN - 2) + l] = tile[k][l]
	monster = ['                  # ', '#    ##    ##    ###', ' #  #  #  #  #  #   ']
	#print(list2string(image).replace("0", ".").replace("1", "#"))
	print(np.array(image).shape)
	global correctresult
	correctresult = list2string(np.flipud(np.array(image))).replace("0", ".").replace("1", "#")
	water = 0
	for n in range(2):
		for m in range(4):
			for i in range(length - len(monster)):
				for j in range(length - len(monster[0])):
					flag = 1
					for k in range(len(monster)):
						for l in range(len(monster[0])):
							if monster[k][l] == '#' and image[i + k][j + l] == '0':
								flag = 0
								break
						if flag == 0:
							break
					else:
						water = 1
						for k in range(len(monster)):
							for l in range(len(monster[0])):
								if monster[k][l] == '#':
									print(i + k, j + l)
									image[i + k][j + l] = '0'
					
			if water == 1:
				
				rough = 0
				for i in range(length):
					for j in range(length):
						if image[i][j] == '1':
							rough += 1
				print(rough)
				raise SystemExit
			image = rotate(image)
		image = flip(image)
	
		
def puzzle(square, avail, x, y):
	possibs = avail.copy()
	if x > 0:
		possibs &= tiles[square[y][x - 1]['tile']][10]
	if y > 0:
		possibs &= tiles[square[y - 1][x]['tile']][10]
	for i in possibs:
		new_avail = avail.copy()
		new_avail.remove(i)
		square[y][x]['tile'] = i
		for j in range(8):
			if x > 0:
				if tiles[i][j][3] != tiles[square[y][x - 1]['tile']][square[y][x - 1]['orient']][1]:
					continue
			if y > 0:
				if tiles[i][j][0] != tiles[square[y - 1][x]['tile']][square[y - 1][x]['orient']][2]:
					continue
			square[y][x]['orient'] = j
			if x == TNUM - 1 and y == TNUM - 1:
				#print(tiles[square[0][0]['tile']][9] * tiles[square[0][TNUM - 1]['tile']][9] * tiles[square[TNUM - 1][0]['tile']][9] * tiles[square[TNUM - 1][TNUM - 1]['tile']][9])
				analyze(square)
			else:
				puzzle(square.copy(), new_avail, 0 if x == TNUM - 1 else x + 1, y + 1 if x == TNUM - 1 else y)
		square[y][x] = {}

square = [[{} for i in range(TNUM)] for i in range(TNUM)]				
avail = set(i for i in range(len(tiles)))
puzzle(square, avail, 0, 0)

(96, 96)
4 56
5 38
5 43
5 44
5 49
5 50
5 55
5 56
5 57
6 39
6 42
6 45
6 48
6 51
6 54
5 30
6 12
6 17
6 18
6 23
6 24
6 29
6 30
6 31
7 13
7 16
7 19
7 22
7 25
7 28
10 66
11 48
11 53
11 54
11 59
11 60
11 65
11 66
11 67
12 49
12 52
12 55
12 58
12 61
12 64
15 21
16 3
16 8
16 9
16 14
16 15
16 20
16 21
16 22
17 4
17 7
17 10
17 13
17 16
17 19
19 62
20 44
20 49
20 50
20 55
20 56
20 61
20 62
20 63
21 45
21 48
21 51
21 54
21 57
21 60
26 92
27 74
27 79
27 80
27 85
27 86
27 91
27 92
27 93
28 75
28 78
28 81
28 84
28 87
28 90
29 61
30 43
30 48
30 49
30 54
30 55
30 60
30 61
30 62
31 44
31 47
31 50
31 53
31 56
31 59
39 26
40 8
40 13
40 14
40 19
40 20
40 25
40 26
40 27
41 9
41 12
41 15
41 18
41 21
41 24
47 60
48 42
48 47
48 48
48 53
48 54
48 59
48 60
48 61
49 43
49 46
49 49
49 52
49 55
49 58
52 80
53 62
53 67
53 68
53 73
53 74
53 79
53 80
53 81
54 63
54 66
54 69
54 72
54 75
54 78
56 22
57 4
57 9
57 10
57 15
57 16
57 21
57 22
57 23
58 5
58 8
58 11
58 14
58 17
58 20
63 40
64 22
64 27
64 28
64 33
64 34
64 39


SystemExit: 

In [321]:
2171-1946

225

In [282]:
for i in range(len(ownresult)):
    if ownresult[i] != correctresult[i]:
        print(i // 96, i % 96)
        if i > 111111111:
            break
        pass

In [349]:
print(correctresult.splitlines()[26:29])
print(ownresult.splitlines()[26:29])
print(correctresult.count('#'))
print(ownresult.count('#'))

m1 = re.finditer(r"(?=(" + kernel_re[1] + "))", ownresult.splitlines()[27])
m2 = re.finditer(r"(?=(" + kernel_re[2] + "))", correctresult.splitlines()[28])
next(m1)
next(m2), next(m2), next(m2)

['...............#..###........##..##.#..##.....##.#.#.#....#.....#......#....#...............##..', '....#.###.........##.#..###....##..##.....#...........#......#........##..#....##....##.#..###..', '.#.#.....#....#.#...#.......#..#..#.........#.......#...#..##.####...#..#..#.##..#.##..#..#...##']
['...............#..###........##..##.#..##.....##.#.#.#....#.....#......#....#...............##..', '....#.###.........##.#..###....##..##.....#...........#......#........##..#....##....##.#..###..', '.#.#.....#....#.#...#.......#..#..#.........#.......#...#..##.####...#..#..#.##..#.##..#..#...##']
2171
2171


(<re.Match object; span=(68, 68), match=''>,
 <re.Match object; span=(71, 71), match=''>,
 <re.Match object; span=(74, 74), match=''>)